In [1]:
import pygame, sys
from pygame.locals import *
import time
import math

pygame.font.init()

WHITE    = (255, 255, 255)
BLUE     = (  0,   0, 255)
RED      = (255,   0,   0)
GREEN    = (0,   255,   0)
BLACK    = (  0,   0,   0)
GOLD     = (255, 215,   0)
HIGH     = (160, 190, 255)

NORTHWEST = "northwest"
NORTHEAST = "northeast"
SOUTHWEST = "southwest"
SOUTHEAST = "southeast"

n=0

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Piece:
        def __init__(self, color):
            self.color = color
            
class Square:
        def __init__(self, color, occupant = None):
            self.color = color 
            self.occupant = occupant


In [3]:
class Graphics:
        def __init__(self):
            self.caption = "CHECKERS ROHIT RANA"

            self.fps = 60
            self.clock = pygame.time.Clock()

            self.window_size = 600
            self.screen = pygame.display.set_mode((self.window_size, self.window_size))


            self.square_size = self.window_size / n
            self.piece_size = self.square_size / 2

            self.message = False
            
            
        def setup_window(self):
            pygame.init()
            pygame.display.set_caption(self.caption)
            
        def board_coords(self,pixel):
            x = int(pixel[0] / self.square_size)
            y = int(pixel[1] / self.square_size)
            return (x,y)
        
        def pixel_coords(self, board_coords):
            x = int(board_coords[0] * self.square_size + self.piece_size+2)
            y = int(board_coords[1] * self.square_size + self.piece_size+2)
            return (x,y)
        
        def update_display(self, board, legal_moves, selected_piece):


            self.draw_board_squares(board)
            self.highlight_squares(legal_moves, selected_piece)
            self.draw_board_pieces(board)

            if self.message:
                self.screen.blit(self.text_surface_obj, self.text_rect_obj)

            pygame.display.update()
            self.clock.tick(self.fps)
            
        def coupdate(self,board):

            self.draw_board_squares(board)
            self.draw_board_pieces(board)
            if self.message:
                self.screen.blit(self.text_surface_obj, self.text_rect_obj)

            pygame.display.update()
            self.clock.tick(self.fps)

        
        def draw_board_squares(self, board):
            for x in range(n):
                for y in range(n):
                    pygame.draw.rect(self.screen, board.matrix[x][y].color, (x * self.square_size, y * self.square_size, self.square_size, self.square_size),0 )

        def draw_board_pieces(self, board):
            for x in range(n):
                for y in range(n):
                    if (board.matrix[x][y].occupant != None):
                        pygame.draw.circle(self.screen, board.matrix[x][y].occupant.color, self.pixel_coords([x,y]), int(self.piece_size-6))


        def highlight_squares(self, squares, origin):
                    for square in squares:
                        if origin[1] - square[1]==1:
                                pygame.draw.rect(self.screen, HIGH, (square[0] * self.square_size, square[1] * self.square_size, self.square_size, self.square_size),5)
                        else:
                                pygame.draw.rect(self.screen, GREEN, (square[0] * self.square_size, square[1] * self.square_size, self.square_size, self.square_size),5)

                    if origin != None:
                        pygame.draw.rect(self.screen, HIGH, (origin[0] * self.square_size, origin[1] * self.square_size, self.square_size, self.square_size))

        def draw_message(self, message):
            self.message = True
            self.font_obj = pygame.font.Font('freesansbold.ttf', 44)
            self.text_surface_obj = self.font_obj.render(message, True, HIGH, BLACK)
            self.text_rect_obj = self.text_surface_obj.get_rect()
            self.text_rect_obj.center = (self.window_size / 2, self.window_size / 2)


In [4]:
class Board:
        def __init__(self):
            self.matrix = self.new_board()

        def new_board(self):
            matrix = [[None] * n for i in range(n)]

            
            for x in range(n):
                for y in range(n):
                    if (y % 2 != 0) and (x % 2 == 0):
                        matrix[x][y] = Square(WHITE)
                    elif (y % 2 != 0) and (x % 2 != 0):
                        matrix[x][y] = Square(BLACK)
                    elif (y % 2 == 0) and (x % 2 != 0):
                        matrix[x][y] = Square(WHITE)
                    elif (y % 2 == 0) and (x % 2 == 0): 
                        matrix[x][y] = Square(BLACK)


            for x in range(n):
                for y in range(math.floor(n/4)):
                    if matrix[x][y].color == BLACK:
                        matrix[x][y].occupant = Piece(RED)
                for y in range(math.ceil(3*n/4), n):
                    if matrix[x][y].color == BLACK:
                        matrix[x][y].occupant = Piece(BLUE)

            return matrix
        
 

        def rel(self,dir,pos):
            x=pos[0]
            y=pos[1]
            
            
            if dir == NORTHWEST:
                return (x - 1, y - 1)
            elif dir == NORTHEAST:
                return (x + 1, y - 1)
            elif dir == SOUTHWEST:
                return (x - 1, y + 1)
            elif dir == SOUTHEAST:
                return (x + 1, y + 1)
            else:
                return 0

        def adjacent(self,pos):
            return [self.rel(NORTHWEST, pos), self.rel(NORTHEAST, pos),self.rel(SOUTHWEST, pos),self.rel(SOUTHEAST, pos)]

        def location(self,pos):
            return self.matrix[pos[0]][pos[1]]

        def blind_legal_moves(self,pos):
            
            
    
            if self.matrix[pos[0]][pos[1]].occupant != None:

                if self.matrix[pos[0]][pos[1]].occupant.color == BLUE:
                    blind_legal_moves = [self.rel(NORTHWEST, pos), self.rel(NORTHEAST, pos)]

                elif self.matrix[pos[0]][pos[1]].occupant.color == RED:
                    blind_legal_moves = [self.rel(SOUTHWEST, pos), self.rel(SOUTHEAST, pos)]

                
            else:
                blind_legal_moves = []

            return blind_legal_moves

        def legal_moves(self,pos):

            blind_legal_moves = self.blind_legal_moves(pos) 
            legal_moves = []
            fmove=[0,0]
            

            for move in blind_legal_moves:
                    
                fmove[0]=move[0] + (move[0] - pos[0])
                fmove[1]=move[1] + (move[1] - pos[1])
                    

                if (self.on_board(move)):
                    if (self.location(move).occupant == None):
                        legal_moves.append(move)

                    elif (self.location(move).occupant.color != self.location(pos).occupant.color and self.on_board(fmove) and self.location(fmove).occupant == None): # is this location filled by an enemy piece?
                        legal_moves.append((fmove[0],fmove[1]))
                                
                fmove=[0,0]
                    


            return legal_moves

        def remove_piece(self,pos):
            self.matrix[int(pos[0])][int(pos[1])].occupant = None

        def move_piece(self,start,end):
            self.matrix[end[0]][end[1]].occupant = self.matrix[start[0]][start[1]].occupant
            self.remove_piece(start)

        def is_end_square(self, coords):
            if coords[1] == 0 or coords[1] == n-1:
                return True
            else:
                return False

        def on_board(self, pos):
            if pos[0] < 0 or pos[1] < 0 or pos[0] > n-1 or pos[1] > n-1:
                return False
            else:
                return True


In [5]:
class Game:

        def __init__(self):
            self.graphics = Graphics()
            self.board = Board()

            self.turn = BLUE
            self.selected_piece = None 
            self.selected_legal_moves = []

        def setup(self):
            self.graphics.setup_window()

        def event_loop(self):
            sel=[0,0]
            self.mouse_pos = self.graphics.board_coords(pygame.mouse.get_pos())

            if self.selected_piece != None:
                self.selected_legal_moves = self.board.legal_moves(self.selected_piece)
                

            for event in pygame.event.get():

                if event.type == QUIT:
                    self.terminate_game()

                if event.type == MOUSEBUTTONDOWN:
                    
    
                    if self.board.location(self.mouse_pos).occupant != None and self.board.location(self.mouse_pos).occupant.color == self.turn:
                        self.selected_piece = self.mouse_pos
                            

                    elif self.selected_piece != None and self.mouse_pos in self.selected_legal_moves:

                            
                        self.board.move_piece(self.selected_piece, self.mouse_pos)
                            

                        if self.mouse_pos not in self.board.adjacent(self.selected_piece):
                            sel[0]=self.selected_piece[0] + (self.mouse_pos[0] - self.selected_piece[0]) / 2
                            sel[1]=self.selected_piece[1] + (self.mouse_pos[1] - self.selected_piece[1]) / 2

                            self.board.remove_piece(sel)

                        self.end_turn()



        def update(self):
            self.graphics.update_display(self.board, self.selected_legal_moves, self.selected_piece)
            

        def terminate_game(self):
            pygame.quit()
            sys.exit

        def main(self):
            self.setup()

            while True:
                self.event_loop()
                self.update()
                
                if self.turn==RED:
                    time.sleep(1)
                    
                    self.computer()
                    
                
                

        def end_turn(self):
            if self.turn == BLUE:
                self.turn = RED
            else:
                self.turn=BLUE
            
            self.selected_piece = None
            self.selected_legal_moves = []

                
            if self.check_for_endgame():
                if self.turn == BLUE:
                    self.graphics.draw_message("COMPUTER WINS!")
                else:
                    self.graphics.draw_message("YOU WINS!")

        def computer(self):
 
                
                for x in range(n):
                    for y in range(n):
                        pos=x,y
                        self.selected_piece=pos


                        if self.board.location(self.selected_piece).occupant != None and self.board.location(self.selected_piece).occupant.color == self.turn:

                            self.selected_legal_moves = self.board.legal_moves(self.selected_piece)

                            for move in self.selected_legal_moves:

                                if move not in self.board.adjacent(self.selected_piece):
                                    oppo1=[0,0]
                                    oppo2=[0,0]
                                    oppo1[0]=move[0]-1
                                    oppo1[1]=move[1]+1
                                    oppo2[0]=move[0]+1
                                    oppo2[1]=move[1]+1


                                    if self.board.on_board(oppo1) and self.board.location(oppo1).occupant != None and self.board.location(oppo1).occupant.color == BLUE:

                                        pass

                                    elif self.board.on_board(oppo2) and self.board.location(oppo2).occupant != None and self.board.location(oppo2).occupant.color == BLUE:

                                        pass   

                                    else:

                                        sel=[0,0]
                                        sel[0]=self.selected_piece[0] + (move[0] - self.selected_piece[0]) / 2
                                        sel[1]=self.selected_piece[1] + (move[1] - self.selected_piece[1]) / 2

                                        self.board.move_piece(self.selected_piece, move)
                                        self.board.remove_piece(sel)
                                        self.graphics.coupdate(self.board)
                                        self.end_turn()
                                        return


                for x in range(n):
                    for y in range(n):
                        pos=x,y
                        self.selected_piece=pos
                    
                        if self.board.location(self.selected_piece).occupant != None and self.board.location(self.selected_piece).occupant.color == self.turn:
                    
                            self.selected_legal_moves = self.board.legal_moves(self.selected_piece)
                            
                            for move in self.selected_legal_moves:
                                
                                if move in self.board.adjacent(self.selected_piece):
                                    oppo1=[0,0]
                                    oppo2=[0,0]
                                    oppo1[0]=move[0]-1
                                    oppo1[1]=move[1]+1
                                    oppo2[0]=move[0]+1
                                    oppo2[1]=move[1]+1

                                    if self.board.on_board(oppo1) and self.board.location(oppo1).occupant != None and self.board.location(oppo1).occupant.color == BLUE:

                                        pass

                                    elif self.board.on_board(oppo2) and self.board.location(oppo2).occupant != None and self.board.location(oppo2).occupant.color == BLUE:

                                        pass   

                                    else:


                                            self.board.move_piece(self.selected_piece, move)

                                            self.graphics.coupdate(self.board)
                                            self.end_turn()
                                            return

                for x in range(n):
                    for y in range(n):
                        pos=x,y
                        self.selected_piece=pos
                        if self.board.location(self.selected_piece).occupant != None and self.board.location(self.selected_piece).occupant.color == self.turn:
                            self.selected_legal_moves = self.board.legal_moves(self.selected_piece)
                            for move in self.selected_legal_moves: 
                                if move not in self.board.adjacent(self.selected_piece):
                                        sel=[0,0]
                                        sel[0]=self.selected_piece[0] + (move[0] - self.selected_piece[0]) / 2
                                        sel[1]=self.selected_piece[1] + (move[1] - self.selected_piece[1]) / 2

                                        self.board.move_piece(self.selected_piece, move)
                                        self.board.remove_piece(sel)
                                        self.graphics.coupdate(self.board)
                                        self.end_turn()
                                        return
                                
                                
                for x in range(n):
                    for y in range(n):
                        pos=x,y
                        self.selected_piece=pos
                    
                        if self.board.location(self.selected_piece).occupant != None and self.board.location(self.selected_piece).occupant.color == self.turn:
                    
                            self.selected_legal_moves = self.board.legal_moves(self.selected_piece)
                            
                            for move in self.selected_legal_moves:
                                self.board.move_piece(self.selected_piece, move)

                                self.graphics.coupdate(self.board)
                                self.end_turn()
                                return
                        

            
        
        def check_for_endgame(self):
            for x in range(n):
                for y in range(n):
                    pos=x,y
                    if self.board.location(pos).color == BLACK and self.board.location(pos).occupant != None and self.board.location(pos).occupant.color == self.turn:
                        if self.board.legal_moves(pos) != []:
                            return False

            return True


In [ ]:
if __name__ == "__main__":
        n=int(input("Enter the Size of the Board "))
        game = Game()
        game.main()


Enter the Size of the Board 8
